In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

import plotly.express as px

## Image Text Retrieval

In [ ]:
uniform_results_path = "./uniform_results/image_text_retrieval/blip2_flickr_results.csv"
df_uniform = pd.read_csv(uniform_results_path)
df_uniform = df_uniform.dropna(axis=1, how="all")

# with pd.option_context('display.max_rows', 10):
#     display(df_uniform)

In [ ]:
awq_results_path = "./awq_results/image_text_retrieval/awq_image_text_retrieval.csv"
df_awq = pd.read_csv(awq_results_path)
# bits --> megabytes
df_awq["model_size"] = df_awq["model_size"] / 8e6

# df_awq['vit_bits'] = df_awq['vit_bits'].astype(str)
# df_awq['qformer_bits'] = df_awq['qformer_bits'].astype(str)

df_awq[:5]

In [ ]:
fig = px.scatter_3d(
    df_awq,
    x="img_r1",
    y="txt_r1",
    z="model_size",
    color="vit_bits",
    symbol="qformer_bits",
    hover_data=list(df_awq.columns),
    title="AWQ Blip-2 Flickr30k Information Retrieval",
)


fig.update_traces(marker=dict(size=4.5))

fig.update_layout(
    scene=dict(
        xaxis_title="Text-->Img R@1",
        yaxis_title="Img -->Text R@1",
        zaxis_title="Model Size (MB)",
        xaxis=dict(autorange="reversed"),
        yaxis=dict(autorange="reversed"),
    ),
)

fig.update_layout(coloraxis_colorbar=dict(yanchor="top", y=1, x=0, ticks="outside"))

fig.write_html("awq_retrieval.html")

fig.show()

In [ ]:
df_awq.sort_values(by="txt_r1", ascending=False).iloc[0]

In [ ]:
df_awq.sort_values(by="img_r1", ascending=False).iloc[0]

In [ ]:
df_awq[(df_awq.vit_bits == 4.0) & (df_awq.qformer_bits == 4.0)]

## Captioning

In [ ]:
awq_results_path = "./awq_image_captioning.csv"
gptq_results_path = "../gptq_scores.csv"

df_awq = pd.read_csv(awq_results_path)
# bits --> megabytes
df_awq["model_size"] = df_awq["model_size"] / 8e6
df_awq = df_awq.rename({"METEOR": "meteor", "CIDEr": "cider"}, axis=1)
df_awq["quant_type"] = "AWQ"


df_awq[:5]

In [ ]:
df_gptq = pd.read_csv(gptq_results_path)
df_gptq["quant_type"] = "GPTQ"
df_gptq[:5]

In [ ]:
df_captioning = pd.merge(df_awq, df_gptq, how="outer")

In [ ]:
df_captioning[:5]

In [ ]:
df_captioning.sort_values(by="meteor", ascending=False).iloc[0]

In [ ]:
df_captioning[(df_captioning.vit_bits == 3) & (df_captioning.qformer_bits == 16) & (df_captioning.llm_bits == 16)]

In [ ]:
df_captioning[(df_captioning.vit_bits == 3) & (df_captioning.qformer_bits == 4) & (df_captioning.llm_bits == 4)]

In [ ]:
df_captioning.sort_values(by="cider", ascending=False).iloc[0]

In [ ]:
df_captioning[(df_captioning.vit_bits == 6) & (df_captioning.qformer_bits == 7) & (df_captioning.llm_bits == 6)]

In [ ]:
hover_data = list(df_awq.columns)
# hover_data.remove('METEOR_per_caption')
# hover_data.remove('CIDEr_per_caption')
hover_data

In [ ]:
fig = px.scatter_3d(
    df_awq,
    x="METEOR",
    y="CIDEr",
    z="model_size",
    color="llm_bits",
    symbol="qformer_bits",
    hover_data=hover_data,
    title="AWQ Blip-2 COCO Captioning",
)


fig.update_traces(marker=dict(size=3.5))

fig.update_layout(
    scene=dict(
        xaxis_title="METEOR",
        yaxis_title="CIDEr",
        zaxis_title="Model Size (MB)",
        xaxis=dict(autorange="reversed"),
        yaxis=dict(autorange="reversed"),
    ),
)

fig.update_layout(coloraxis_colorbar=dict(yanchor="top", y=1, x=0, ticks="outside"))

fig.write_html("awq_captioning.html")

fig.show()

In [ ]:
hover_data = list(df_captioning.columns)

fig = px.scatter(
    df_captioning,
    x="meteor",
    y="cider",
    color="quant_type",
    hover_data=hover_data,
    title="Blip-2 COCO Captioning",
)

fig.update_traces(marker=dict(line=dict(width=2, color="DarkSlateGrey")), opacity=0.75)

fig.write_html("coco_captioning.html")

fig.show()